### Fake News Classifier using LSTM

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("data/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df=df.dropna()

In [4]:
X=df.drop('label', axis=1)
X.shape

(18285, 4)

In [6]:
y=df['label']
y.shape

(18285,)

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [9]:
## Vocabulary size
voc_size=5000

### One hot representation

In [10]:
messages=X.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\736944\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### Data Preprocessing

In [18]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
    print(i)
    review=re.sub('[^a-zA-Z]', ' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    
    review= [ps.stem(word) for word in review if not word in stopwords.words('english') ]
    print(review)
    review=' '.join(review)
    print(review)
    corpus.append(review)
    

0
['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet']
hous dem aid even see comey letter jason chaffetz tweet
1
['flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart']
flynn hillari clinton big woman campu breitbart
2
['truth', 'might', 'get', 'fire']
truth might get fire
3
['civilian', 'kill', 'singl', 'us', 'airstrik', 'identifi']
civilian kill singl us airstrik identifi
4
['iranian', 'woman', 'jail', 'fiction', 'unpublish', 'stori', 'woman', 'stone', 'death', 'adulteri']
iranian woman jail fiction unpublish stori woman stone death adulteri
5
['jacki', 'mason', 'hollywood', 'would', 'love', 'trump', 'bomb', 'north', 'korea', 'lack', 'tran', 'bathroom', 'exclus', 'video', 'breitbart']
jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart
6
['beno', 'hamon', 'win', 'french', 'socialist', 'parti', 'presidenti', 'nomin', 'new', 'york', 'time']
beno hamon win french socialist parti presidenti nomi

['fox', 'biz', 'report', 'help', 'bash', 'clinton', 'ralli', 'cover', 'trump', 'pack', 'event', 'day']
fox biz report help bash clinton ralli cover trump pack event day
78
['fiction', 'podcast', 'worth', 'listen', 'new', 'york', 'time']
fiction podcast worth listen new york time
79
['mike', 'birbiglia', 'tip', 'make', 'small', 'hollywood', 'anywher', 'new', 'york', 'time']
mike birbiglia tip make small hollywood anywher new york time
80
['invest', 'strategist', 'forecast', 'collaps', 'timelin', 'last', 'gasp', 'econom', 'cycl', 'come']
invest strategist forecast collaps timelin last gasp econom cycl come
81
['venezuela', 'muzzl', 'legislatur', 'move', 'closer', 'one', 'man', 'rule', 'new', 'york', 'time']
venezuela muzzl legislatur move closer one man rule new york time
82
['whether', 'john', 'mccain', 'mitt', 'romney', 'donald', 'trump', 'democrat', 'alway', 'run', 'war', 'women', 'tactic', 'destroy', 'republican', 'candid']
whether john mccain mitt romney donald trump democrat alway 

['va', 'fail', 'properli', 'examin', 'thousand', 'veteran']
va fail properli examin thousand veteran
152
['trump', 'famili', 'alreadi', 'sworn', 'secreci', 'fake', 'moon', 'land', 'soon']
trump famili alreadi sworn secreci fake moon land soon
153
['sport', 'writer', 'nfl', 'great', 'jim', 'brown', 'decad', 'civil', 'right', 'work', 'eras', 'say', 'nice', 'thing', 'donald', 'trump', 'breitbart']
sport writer nfl great jim brown decad civil right work eras say nice thing donald trump breitbart
154
['watch', 'tv', 'excus', 'republican', 'skip', 'donald', 'trump', 'convent', 'new', 'york', 'time']
watch tv excus republican skip donald trump convent new york time
155
['open', 'letter', 'trump', 'voter', 'told', 'like']
open letter trump voter told like
156
['comment', 'power', 'corpor', 'lobbi', 'quietli', 'back', 'hillari', 'nobodi', 'talk', 'runsinquicksand']
comment power corpor lobbi quietli back hillari nobodi talk runsinquicksand
157
['hijack', 'end', 'peac', 'libyan', 'airlin', 'land

['leader', 'applaud', 'gorsuch', 'confirm', 'win', 'pro', 'life', 'movement']
leader applaud gorsuch confirm win pro life movement
220
['newstick']
newstick
221
['french', 'vogu', 'march', 'cover', 'featur', 'transgend', 'model', 'new', 'york', 'time']
french vogu march cover featur transgend model new york time
222
['trump', 'veer', 'parti', 'line', 'gun', 'control', 'new', 'york', 'time']
trump veer parti line gun control new york time
223
['oligarchi', 'prepar', 'groundwork', 'steal', 'elect']
oligarchi prepar groundwork steal elect
224
['ya', 'hay', 'reencuentro', 'de', 'operaci', 'n', 'triunfo', 'que', 'edicion', 'de', 'operaci', 'n', 'triunfo']
ya hay reencuentro de operaci n triunfo que edicion de operaci n triunfo
225
['chatsworth', 'hous', 'tale', 'five', 'centuri', 'new', 'york', 'time']
chatsworth hous tale five centuri new york time
226
['uncomfort', 'love', 'affair', 'donald', 'trump', 'new', 'england', 'patriot', 'new', 'york', 'time']
uncomfort love affair donald trump n

['million', 'american', 'kill', 'minut']
million american kill minut
300
['wayn', 'madsen', 'cia', 'alway', 'serv', 'interest', 'wall', 'street']
wayn madsen cia alway serv interest wall street
301
['novemb', 'daili', 'contrarian', 'read']
novemb daili contrarian read
302
['flashback', 'report', 'obama', 'campaign', 'rep', 'talk', 'iran', 'hama']
flashback report obama campaign rep talk iran hama
303
['lesseroftwoevil']
lesseroftwoevil
304
['aya', 'cash', 'first', 'time', 'ate', 'veget', 'new', 'york', 'time']
aya cash first time ate veget new york time
305
['trump', 'advis', 'say', 'isra', 'settlement', 'illeg']
trump advis say isra settlement illeg
306
['serena', 'william', 'prevail', 'open', 'problem', 'new', 'york', 'time']
serena william prevail open problem new york time
307
['soul', 'man', 'sam', 'moor', 'honor', 'perform', 'trump', 'inaugur']
soul man sam moor honor perform trump inaugur
308
['ferrel', 'came', 'back', 'bush', 'destroy', 'trump', 'video']
ferrel came back bush d

...

17967
['bypass', 'stigma', 'take', 'care', 'daughter', 'new', 'york', 'time']
bypass stigma take care daughter new york time
17968
['epic', 'new', 'yorker', 'hillari', 'theme', 'haunt', 'hous', 'lib', 'absolut', 'furiou']
epic new yorker hillari theme haunt hous lib absolut furiou
17969
['view', 'syria', 'ceas', 'fire', 'take', 'effect', 'new', 'york', 'time']
view syria ceas fire take effect new york time
17970
['mistrial', 'south', 'carolina', 'offic', 'shot', 'walter', 'scott', 'new', 'york', 'time']
mistrial south carolina offic shot walter scott new york time
17971
['asteroid', 'warn', 'system', 'found', 'citi', 'killer', 'object', 'head', 'toward', 'earth']
asteroid warn system found citi killer object head toward earth
17972
['conway', 'see', 'hyster', 'democrat', 'parti', 'unravel', 'front', 'eye', 'breitbart']
conway see hyster democrat parti unravel front eye breitbart
17973
['despit', 'cancer', 'diagnosi', 'husband', 'death', 'keep', 'posit', 'outlook', 'new', 'york', 'time'

['watch', 'fast', 'msnbc', 'cut', 'black', 'woman', 'interview', 'say', 'trump']
watch fast msnbc cut black woman interview say trump
18043
['david', 'brook', 'white', 'vote', 'gene', 'pool', 'ruin', 'global', 'immigr', 'femin']
david brook white vote gene pool ruin global immigr femin
18044
['nick', 'cannon', 'plan', 'parenthood', 'found', 'extermin', 'black', 'race']
nick cannon plan parenthood found extermin black race
18045
['marvel', 'artist', 'fire', 'hide', 'anti', 'semit', 'messag', 'x', 'men', 'comic', 'breitbart']
marvel artist fire hide anti semit messag x men comic breitbart
18046
['presid', 'trump', 'quietli', 'roll', 'back', 'obama', 'era', 'gun', 'control']
presid trump quietli roll back obama era gun control
18047
['crook', 'hillari', 'risk', 'blue', 'dress', 'moment', 'aggress', 'attack', 'fbi', 'jame', 'comey']
crook hillari risk blue dress moment aggress attack fbi jame comey
18048
['met', 'opera', 'struggl', 'fill', 'empti', 'seat', 'new', 'york', 'time']
met opera 

['dear', 'america', 'let', 'unit', 'flood', 'nation', 'optim']
dear america let unit flood nation optim
18114
['ravish', 'kumar', 'prime', 'time', 'show', 'nomin', 'filmfar', 'award', 'best', 'drama', 'seri']
ravish kumar prime time show nomin filmfar award best drama seri
18115
['clinton', 'polici', 'syria', 'lead', 'ww', 'say', 'trump']
clinton polici syria lead ww say trump
18116
['femal', 'scientist', 'turn', 'data', 'fight', 'lack', 'represent', 'panel', 'new', 'york', 'time']
femal scientist turn data fight lack represent panel new york time
18117
['e', 'pasaport', 'por', 'domingo', 'para', 'la', 'televisi', 'n', 'por', 'herbert', 'mujica', 'roja']
e pasaport por domingo para la televisi n por herbert mujica roja
18118
['usa', 'continu', 'drive', 'plutocraci', 'dismantl', 'state']
usa continu drive plutocraci dismantl state
18119
['conven', 'grand', 'juri', 'trust', 'trump', 'surrog', 'lose', 'clinton', 'email', 'video']
conven grand juri trust trump surrog lose clinton email vid

['comment', 'donald', 'trump', 'replac', 'filipina', 'white', 'hous', 'chef', 'paula', 'deen', 'mitch', 'jalandoni']
comment donald trump replac filipina white hous chef paula deen mitch jalandoni
18188
['comment', 'strang', 'fact', 'presid', 'tania', 'de', 'saram']
comment strang fact presid tania de saram
18189
['outsid', 'select', 'trump', 'aim', 'unnerv', 'washington', 'new', 'york', 'time']
outsid select trump aim unnerv washington new york time
18190
['green', 'vacat', 'new', 'york', 'time']
green vacat new york time
18191
['exclus', 'front', 'nation', 'candid', 'reveal', 'parti', 'potenti', 'new', 'name']
exclus front nation candid reveal parti potenti new name
18192
['leak', 'email', 'reveal', 'hillari', 'speak', 'long', 'without', 'podium', 'debat', 'rig']
leak email reveal hillari speak long without podium debat rig
18193
['top', 'radio', 'host', 'michael', 'savag', 'invad', 'right']
top radio host michael savag invad right
18194
['kim', 'davi', 'could', 'pay', 'much', 'legal

['samsung', 'kim', 'jong', 'un', 'rex', 'tillerson', 'morn', 'brief', 'new', 'york', 'time']
samsung kim jong un rex tillerson morn brief new york time
18270
['comment', 'world', 'heav', 'sigh', 'relief', 'trump', 'victori', 'debbi', 'menon']
comment world heav sigh relief trump victori debbi menon
18271
['ann', 'coulter', 'provid', 'univers', 'health', 'care', 'use', 'one', 'easi', 'trick']
ann coulter provid univers health care use one easi trick
18272
['salli', 'yate', 'say', 'trump', 'wiretap', 'breitbart']
salli yate say trump wiretap breitbart
18273
['main', 'gov', 'lepag', 'threaten', 'investig', 'colleg', 'student', 'fraud', 'vote', 'video']
main gov lepag threaten investig colleg student fraud vote video
18274
['sen', 'mcconnel', 'suprem', 'court', 'vacanc', 'key', 'presid', 'trump', 'win', 'breitbart']
sen mcconnel suprem court vacanc key presid trump win breitbart
18275
['nikki', 'haley', 'blast', 'u', 'n', 'human', 'right', 'offic', 'unfair', 'bia', 'israel', 'latest', 'rep

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
one_hot_repr=[one_hot(words, voc_size) for words in corpus]

In [22]:
one_hot_repr

[[1293, 2930, 4252, 3709, 151, 2815, 1602, 1890, 1128, 859],
 [3095, 1808, 2843, 3807, 4909, 2118, 1679],
 [4673, 3022, 1114, 3952],
 [744, 3721, 612, 3666, 1551, 214],
 [1701, 4909, 299, 2165, 1520, 1129, 4909, 4008, 738, 840],
 [3826,
  1194,
  2500,
  4520,
  296,
  1701,
  4072,
  1487,
  2932,
  294,
  3447,
  2789,
  2476,
  2320,
  1679],
 [2877, 3264, 2746, 4700, 4457, 4871, 1108, 2053, 4253, 2052, 3020],
 [2286, 3798, 1264, 3544, 4214, 4545, 1701, 4097, 4253, 2052, 3020],
 [508, 4378, 2542, 3732, 3697, 3178, 4208, 4413, 1701, 4215],
 [631, 3844, 4630, 4182, 2165, 1951, 792, 833],
 [1995, 189, 1910, 1385, 317, 2820, 3976, 1333, 4955, 1167, 3541],
 [3666, 4336, 151, 3178, 1701, 4214],
 [3548, 3377, 4595, 268, 2141, 879, 2835, 3399, 3699],
 [4569, 2854, 1177, 1307, 4967, 3809, 1892, 4253, 2052, 3020],
 [2820, 1881, 807, 168, 1610, 4253, 2052, 3020],
 [1566, 1210, 507, 1969, 2781, 161, 202, 684, 3225, 4798],
 [1824, 2890, 1808],
 [3156, 4685, 2476, 4031, 1701, 4477, 4116, 1679],
 

### Embedding Representation

In [52]:
sent_length=20
embedded_docs=pad_sequences(one_hot_repr, padding='pre', maxlen=sent_length )
print(embedded_docs)

[[   0    0    0 ... 1890 1128  859]
 [   0    0    0 ... 4909 2118 1679]
 [   0    0    0 ... 3022 1114 3952]
 ...
 [   0    0    0 ... 4253 2052 3020]
 [   0    0    0 ... 1320 3639 3968]
 [   0    0    0 ...  388 1704 1145]]


In [53]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1293,
       2930, 4252, 3709,  151, 2815, 1602, 1890, 1128,  859])

### Creating Model

In [54]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_final,y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 39ms/step - loss: 0.3212 - accuracy: 0.8534 - val_loss: 0.1921 - val_accuracy: 0.9198
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1348 - accuracy: 0.9485 - val_loss: 0.2032 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0935 - accuracy: 0.9654 - val_loss: 0.2343 - val_accuracy: 0.9092
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0688 - accuracy: 0.9769 - val_loss: 0.2803 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0478 - accuracy: 0.9847 - val_loss: 0.3074 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0319 - accuracy: 0.9901 - val_loss: 0.3277 - val_accuracy: 0.9109
Epoch 7/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0193 - accuracy: 0.9942 - val_loss: 0.4125 - val_accuracy: 0.910

### Adding dropout

In [60]:
from tensorflow.keras.layers import Dropout
#creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


### Performance Metrics and Accuracy

In [61]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 3s 9ms/step


In [62]:
from sklearn.metrics import confusion_matrix

In [63]:
confusion_matrix(y_test, y_pred)

array([[2978,  441],
       [1817,  799]], dtype=int64)

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6258492129246065